In [3]:
import pandas as pd
import numpy as np
import config
from sqlalchemy import text
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [23]:
sql = """select * from apl.damir_age_stats 
where (psp_spe_snds > 0 and psp_spe_snds <= 42)
or (psp_act_snds > 0 and psp_act_snds <= 50)
"""
damir = pd.read_sql(sql, config.connection_string)
damir
damir["psp_act_snds"].unique()

array([nan, 19., 21., 24., 26., 27., 28., 29., 30., 50.])

In [8]:
sql =f"""
    select year, sum(pop) pop, sum(pop0002)/sum(pop) p0002, sum(pop0305)/sum(pop) p0305, sum(pop0610)/sum(pop) p0610, sum(pop1117)/sum(pop) p1117, sum(pop1824)/sum(pop) p1824, sum(pop2539)/sum(pop) p2539, sum(pop4054)/sum(pop) p4054, sum(pop5564)/sum(pop) p5564, sum(pop6579)/sum(pop) p6579, sum(pop80p)/sum(pop) p80p
    from iris.pop_iris
    group by year
"""
pop_iris = pd.read_sql(sql, config.connection_string)
pop_iris

,year,pop,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,20,67162154.0,0.032056,0.034700,0.061375,0.086881,0.080607,0.180414,0.195735,0.126311,0.140958,0.060963
1,21,67408052.0,0.031403,0.033951,0.060709,0.087117,0.081043,0.179629,0.194405,0.126668,0.144234,0.060842


In [9]:
pop_iris[pop_iris["year"]==20].iloc[0].to_dict()

{'year': 20.0,
 'pop': 67162153.99999996,
 'p0002': 0.03205633570254434,
 'p0305': 0.03469965383959878,
 'p0610': 0.061374556855643844,
 'p1117': 0.08688149205273843,
 'p1824': 0.08060697124120421,
 'p2539': 0.1804144005357685,
 'p4054': 0.19573528970808554,
 'p5564': 0.12631058481640914,
 'p6579': 0.14095808265358653,
 'p80p': 0.06096263259442096}

In [24]:
def compute_overrepresentation(damir_dataset, year, psp_spe_snds, psp_act_snds):
    yy=min(year,21)
    is_medecin = psp_spe_snds is not None
    props=pop_iris[pop_iris["year"]==yy].iloc[0].to_dict()
    bb = 1 if psp_spe_snds==1 or psp_spe_snds==12 else 0
    damir = damir_dataset[damir_dataset["ann"]==year]
    if is_medecin:
        damir = damir[damir["psp_spe_snds"]==psp_spe_snds]
    else:
        damir = damir[damir["psp_act_snds"]==psp_act_snds]
    overs={"year":year, "psp_spe_snds":psp_spe_snds, "psp_act_snds":psp_act_snds}
    overs["p0002"]=(damir[damir["age_ben_snds"]==0]["ratio"]*(3+bb*3)/20/(props["p0002"])).iloc[0]
    overs["p0305"]=(damir[damir["age_ben_snds"]==0]["ratio"]*3/20/(props["p0305"])).iloc[0]
    overs["p0610"]=(damir[damir["age_ben_snds"]==0]["ratio"]*(5-bb)/20/(props["p0610"])).iloc[0]
    overs["p1117"]=(damir[damir["age_ben_snds"]==0]["ratio"]*(7-bb)/20/(props["p1117"])).iloc[0]
    overs["p1824"]=((damir[damir["age_ben_snds"]==0]["ratio"]*(2-bb)/20+damir[damir["age_ben_snds"]==20]["ratio"].iloc[0]*5/10)/(props["p1824"])).iloc[0]
    overs["p2539"]=((damir[damir["age_ben_snds"]==20]["ratio"]*5/10+damir[damir["age_ben_snds"]==30]["ratio"].iloc[0]*10/10)/(props["p2539"])).iloc[0]
    overs["p4054"]=((damir[damir["age_ben_snds"]==40]["ratio"]*10/10+damir[damir["age_ben_snds"]==50]["ratio"].iloc[0]*5/10)/(props["p4054"])).iloc[0]
    overs["p5564"]=((damir[damir["age_ben_snds"]==50]["ratio"]*5/10+damir[damir["age_ben_snds"]==60]["ratio"].iloc[0]*5/10)/(props["p5564"])).iloc[0]
    overs["p6579"]=((damir[damir["age_ben_snds"]==60]["ratio"]*5/10+damir[damir["age_ben_snds"]==70]["ratio"].iloc[0]*10/10)/(props["p6579"])).iloc[0]
    ratio99 = damir[damir["age_ben_snds"]==99]["ratio"].iloc[0] if len(damir[damir["age_ben_snds"]==99]) > 0 else 0
    overs["p80p"]=((damir[damir["age_ben_snds"]==80]["ratio"]*10/10+ratio99*10/10)/(props["p80p"])).iloc[0]
    if psp_spe_snds == 12:
        overs["p0002"]+=overs["p1117"]
        overs["p0002"]+=overs["p1824"]
        overs["p0002"]+=overs["p2539"]
        overs["p0002"]+=overs["p4054"]
        overs["p0002"]+=overs["p5564"]
        overs["p0002"]+=overs["p6579"]
        overs["p0002"]+=overs["p80p"]
        overs["p1117"]=overs["p1824"]=overs["p2539"]=overs["p4054"]=overs["p5564"]=overs["p6579"]=overs["p80p"]=0
    return overs

# todo Gériatre : Tout mettre > 60 ans
# todo Sage femme : Que des femmes < 50 ans
overs = compute_overrepresentation(damir, 20, None, 28)
overs

{'year': 20,
 'psp_spe_snds': None,
 'psp_act_snds': 28,
 'p0002': 0.7276064550631264,
 'p0305': 0.672179523480558,
 'p0610': 0.6333894145990067,
 'p1117': 0.6264118077099965,
 'p1824': 0.47561397008527956,
 'p2539': 0.34921093204340253,
 'p4054': 0.5889663612137105,
 'p5564': 1.3796368280436602,
 'p6579': 2.6056840323496484,
 'p80p': 1.671133139557698}

In [25]:
def df_append_dict(df, dico):
    dico_df = pd.DataFrame([dico])
    return pd.concat([df, dico_df], ignore_index=True)

df = pd.DataFrame(columns=overs.keys())
df = df_append_dict(df, overs)
df

,year,psp_spe_snds,psp_act_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,20,None,28,0.727606,0.67218,0.633389,0.626412,0.475614,0.349211,0.588966,1.379637,2.605684,1.671133


In [29]:
years=damir["ann"].unique()
print(years)
psp_spe_sndss=damir["psp_spe_snds"].dropna().unique()
psp_act_sndss=damir["psp_act_snds"].dropna().unique()
print(psp_spe_sndss)
psp_act_sndss

[20 21 22 23 24]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 11. 12. 13. 14. 15. 18. 31. 32. 33.
 34. 35. 36. 37. 38. 42.]


array([19., 21., 24., 26., 27., 28., 29., 30., 50.])

In [30]:
df = pd.DataFrame(columns=overs.keys())
for year in years:
    for i in psp_spe_sndss:
        overs = compute_overrepresentation(damir, year, i, None)
        df = df_append_dict(df, overs)
    for i in psp_act_sndss:
        overs = compute_overrepresentation(damir, year, None, i)
        df = df_append_dict(df, overs)
df

,year,psp_spe_snds,psp_act_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,20,1.0,NaN,0.905386,0.418208,0.315260,0.334057,0.610763,0.786168,0.885518,1.133266,1.638287,2.782435
1,20,2.0,NaN,0.359383,0.332007,0.312847,0.309401,0.603956,0.815675,0.985486,1.296469,1.841326,1.949265
2,20,3.0,NaN,0.122765,0.113413,0.106868,0.105691,0.240940,0.376122,0.802847,1.401387,2.448984,3.440615
3,20,4.0,NaN,0.319696,0.295343,0.278299,0.275233,0.604697,0.772066,1.020762,1.355475,1.847434,1.952917
4,20,5.0,NaN,0.579068,0.534957,0.504085,0.498532,0.870782,0.909465,0.880232,1.082622,1.578259,2.015069
...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,24,NaN,27.0,0.641334,0.593200,0.552900,0.539416,0.792564,0.818640,0.812932,1.058675,1.676133,2.202187
161,24,NaN,28.0,2.944279,2.723302,2.538288,2.476383,0.956772,0.224926,0.318002,0.421175,0.875005,1.412389
162,24,NaN,29.0,1.401909,1.296691,1.208598,1.179122,1.478940,1.529559,0.927036,0.557955,0.475441,0.357952
163,24,NaN,30.0,0.261358,0.241742,0.225319,0.219824,0.681282,0.925995,1.080261,1.276241,1.496716,2.328377


In [31]:
df.to_sql("overrepresentation_2", config.connection_string, schema="apl", if_exists="replace", index=False)

165